# Tucker Decomposition using ALS

In [1]:
import torch
import numpy as np
import tensorly as tl # Used for verification

## Mode-N Matricization

In [2]:
"""
Input:  1) tensor: Input tensor
        2) n: mode along which to matricize the tensor (mode is 0-indexed)
Output: matrix: n-mode matricization of the tensor
"""
# Take mode = 0 for the first mode, mode = 1 for the second mode, ...., mode = n-1 for nth mode
def mode_n_matricization(tensor, n):
    mode = n
    # Get the size of the original tensor
    sz = tensor.size()
    # print(f"tensor size: {sz}")

    # Permute the dimensions of the tensor to bring the chosen mode to the front
    # This will make it easy to reshape the tensor into a matrix along the chosen mode
    permuted_dimensions = list(range(len(sz)))
#     print('Before Permuation, dimensions: ', permuted_dimensions)
    permuted_dimensions.remove(mode)
    permuted_dimensions.insert(0, mode)
#     print('After Permuation, dimensions: ', permuted_dimensions)
    permuted_tensor = tensor.permute(*permuted_dimensions)
#     print(f"permuted tensor size: {permuted_tensor.size()}")

    # Reshape the permuted tensor into a matrix along the chosen mode
    matrix = permuted_tensor.reshape(sz[mode], -1)
#     print(f"matrix size: {matrix.size()}")

#     print(f"n-mode matricization along mode {mode}:")
#     print(matrix)

    return matrix

## Test Mode-N Matricization

In [3]:
my_tensor = torch.empty((2,3,4))
my_tensor[0, :, :] = torch.tensor([[1,4,7,10], [2,5,8,11], [3,6,9,12]])
my_tensor[1, :, :] = torch.tensor([[13,16,19,22], [14,17,20,23], [15,18,21,24]])
print(f"Original Tensor: {my_tensor.size()}")

for n in range(len(my_tensor.size())):
    print(f"\nn-mode matricization along mode {n}:")
    matrix = mode_n_matricization(my_tensor, n)
    print(matrix)

Original Tensor: torch.Size([2, 3, 4])

n-mode matricization along mode 0:
tensor([[ 1.,  4.,  7., 10.,  2.,  5.,  8., 11.,  3.,  6.,  9., 12.],
        [13., 16., 19., 22., 14., 17., 20., 23., 15., 18., 21., 24.]])

n-mode matricization along mode 1:
tensor([[ 1.,  4.,  7., 10., 13., 16., 19., 22.],
        [ 2.,  5.,  8., 11., 14., 17., 20., 23.],
        [ 3.,  6.,  9., 12., 15., 18., 21., 24.]])

n-mode matricization along mode 2:
tensor([[ 1.,  2.,  3., 13., 14., 15.],
        [ 4.,  5.,  6., 16., 17., 18.],
        [ 7.,  8.,  9., 19., 20., 21.],
        [10., 11., 12., 22., 23., 24.]])


## Higher Order Singular Value Decomposition (HOSVD)

In [4]:
"""
Input:  1) X: Input tensor of size (s_1,s_2,...,s_N)
        2) R: array of decomposition ranks of size N (R_1,R_2,...,R_N)
Output: A: array of factor matrices of size N (A_1,A_2,...,A_N)
"""
def HOSVD(X, R):
    N = len(R)
    A = []
    for n in range(N):
        # Matricize the input tensor along the n-th mode
        Xn = mode_n_matricization(X, n)
        # Compute the left singular matrix of the n-th matricization
        U = torch.svd(Xn).U
        # Take the leading R[n] columns of the left singular matrix
        if(R[n] > U.size()[1]):
            Zero = torch.zeros(U.size()[0], R[n] - U.size()[1])
            An = torch.cat((U, Zero), dim=1)
        else:
            An = U[:, :R[n]]
        A.append(An)
    return A  
        

## Einsum functions

In [5]:
"""
Input: T: Input tensor
Output: equation: equation string for torch.einsum()
"""
def einsum_equation(T):
    equation = ''
    for i in range(len(T.size())): 
        equation += f'{chr(97 + i)}'
    for i in range(len(T.size())):
        equation += f',{chr(122 -i)}{chr(97 + i)}'
    equation += '->'
    for i in range(len(T.size())):
        equation += f'{chr(122 - i)}'
    # print(f"equation: {equation}")
    return equation

T = torch.randn((2,3,4,5,6))
eqn = einsum_equation(T)
print(f"equation: {eqn}")

equation: abcde,za,yb,xc,wd,ve->zyxwv


## Test HOSVD

In [6]:
my_tensor = torch.randn((2,3,4,5))
ranks = [1,2,3,4]
print(f"Original Tensor: {my_tensor.size()}")

# Decompose the tensor into a core tensor and factor matrices
A = HOSVD(my_tensor, ranks)
A_T = [A[i].T for i in range(len(A))]
for i in range(len(A)):
    print(f"Factor Matrix {i+1}: {A[i].size()}")

eqn = einsum_equation(my_tensor)
G = torch.einsum(eqn, my_tensor, *A_T)
print(f"Core Tensor: {G.size()}")

# Multiply matrices with core tensor to obtain the original tensor
my_tensor_reconstructed = torch.einsum(eqn, G, *A)

# Reshape the reconstructed tensor to the original size
print(f"Reconstructed Tensor: {my_tensor_reconstructed.size()}")

Original Tensor: torch.Size([2, 3, 4, 5])
Factor Matrix 1: torch.Size([2, 1])
Factor Matrix 2: torch.Size([3, 2])
Factor Matrix 3: torch.Size([4, 3])
Factor Matrix 4: torch.Size([5, 4])
Core Tensor: torch.Size([1, 2, 3, 4])
Reconstructed Tensor: torch.Size([2, 3, 4, 5])


## Higher-Order Orthogonal Iteration (HOOI)

In [7]:
"""
Input:  1) T: Input tensor of size (s_1,s_2,...,s_N)
        2) R: array of decomposition ranks of size N (R_1,R_2,...,R_N)
Output: 1) G: core tensor of size (R_1,R_2,...,R_N)
        2) A: array of factor matrices of size N (A_1,A_2,...,A_N)
"""
def Tucker_ALS(T,R, max_iter=100, tol=1e-20):
    # Initialize the factor matrices with HOSVD
    A = HOSVD(T, R)
    A_T = [A[i].T for i in range(len(A))]

    # Iterate until convergence
    for iter in range(max_iter):
        prev_factors = A.copy()

        # Update the factor matrices one by one
        for n in range(len(R)):
            B = A_T.copy()
            B[n] = torch.eye(T.size()[n])
            eqn = einsum_equation(T)
            Y = torch.einsum(eqn, T, *B)

            # Take the leading R[n] columns of the left singular matrix of Yn
            Yn = mode_n_matricization(Y, n)
            U = torch.svd(Yn).U            
            if(R[n] > U.size()[1]):
                Zero = torch.zeros(U.size()[0], R[n] - U.size()[1])
                An = torch.cat((U, Zero), dim=1)
            else:
                An = U[:, :R[n]]

            # Update the factor matrix
            A[n] = An
            A_T[n] = An.T

        # Check for convergence
        if all(torch.norm(A[i] - prev_factors[i]) < tol for i in range(T.dim())):
            print(f"Converged at iteration {iter}")
            break
    
    # Compute the core tensor after Tucker decomposition
    G = torch.einsum(eqn, T, *A_T)

    return G, A           

## Test HOOI

In [8]:
tensor = torch.randn((3,4,5,6))
ranks=[2, 3, 2, 2]
print("Original tensor:")
print(tensor)

"""
Tucker Decomposition using TensorLy
"""
print("Tucker Decomposition using TensorLy:")
# Perform Tucker decomposition
T = tensor.numpy()
core_tensor, factor_matrices =  tl.decomposition.tucker(T, ranks)

# Print the shapes of the core tensor and factor matrices
print("Core Tensor Shape:", core_tensor.shape)
print("Core Tensor:")
print(core_tensor)
for i in range(len(factor_matrices)):
    print("Factor Matrix {} Shape: {}".format(i+1, factor_matrices[i].shape))
    print("Factor Matrix {}: ".format(i+1))
    print(factor_matrices[i])

# Norm difference between original tensor and reconstructed tensor
print("\nNorm difference between original tensor and reconstructed tensor: {}".format(tl.norm(T - tl.tucker_to_tensor((core_tensor, factor_matrices)))))



"""
Tucker Decomposition using Tucker_ALS
"""
print("\nTucker Decomposition using Tucker_ALS:")
G, A = Tucker_ALS(tensor, ranks)

# Print the resulting core tensor and factor matrices
print("Core tensor Shape:", G.shape)
print('Core tensor:')
print(G)
for n in range(len(ranks)):
    print("\nFactor matrix {} Shape: {}".format(n+1, A[n].shape))
    print(f"Factor matrix {n+1}:")
    print(A[n])

# Norm difference between original tensor and reconstructed tensor
eqn = einsum_equation(tensor)
print(f"\nNorm difference between original tensor and reconstructed tensor: {torch.norm(tensor - torch.einsum(eqn, G, *A))}")


Original tensor:
tensor([[[[-0.4289, -0.9889,  0.9181,  0.5114,  0.0364, -0.1731],
          [-0.9026,  0.5471, -0.4279,  0.0301,  0.5567, -0.8803],
          [-0.5848, -0.6082,  0.7001, -1.5437,  0.3230,  0.0702],
          [ 0.0175,  0.9655,  1.0856, -0.7991,  0.5537,  0.2649],
          [ 0.9500,  0.3290,  3.7842, -0.2163, -0.0925, -1.1061]],

         [[-0.4849,  0.0885, -0.3247,  2.2043,  0.6503, -0.4545],
          [ 1.2729, -0.0250, -1.2669,  0.1334,  0.1942,  1.3346],
          [-0.6313,  1.6723,  0.5130, -2.7286,  1.3269, -0.1290],
          [ 0.2781, -0.6621, -1.2806, -1.7114,  0.8791, -1.2371],
          [ 1.0276, -0.8481, -1.0582,  2.0718, -1.5551, -0.5539]],

         [[ 0.2467, -0.2310,  1.3404, -1.4266,  0.1829, -0.2164],
          [-0.6306,  0.6844,  0.5973, -0.6551, -0.4004,  0.7745],
          [ 1.6561, -0.1613,  0.6671,  0.2463, -0.9480, -0.7451],
          [ 0.2740, -0.4210, -0.1016, -0.1012,  1.1087,  0.9888],
          [-1.2599, -0.3694, -1.1233, -0.5488, -0.7771,